In [75]:
import json
import pandas as pd 
from db.redis_database import RedisDatabase

In [32]:
timeseries_data = r.hgetall("MCX:CRUDEOIL21OCTFUT")

columns = ["timecode", "price", "volume"]
rows = []

for timecode, price_vol in timeseries_data.items():
    d = json.loads(price_vol)
    price = d["price"]
    vol = d["volume"]

    rows.append([timecode, price, vol])

df = pd.DataFrame(rows, columns=columns)
df.set_index("timecode", inplace=True)
df.index = pd.to_datetime(df.index)
df.sort_values(by="timecode", inplace=True)

In [47]:
ohlc = df["price"].resample("1min").ohlc()

In [49]:
ohlc["volume_diff"] = (df["volume"].resample("1min").max()).diff()

In [56]:
ohlc

,open,high,low,close,volume_diff
timecode,,,,,
2021-10-15 17:08:00,6152.0,6154.0,6152.0,6153.0,NaN
2021-10-15 17:09:00,6153.0,6155.0,6152.0,6154.0,18.0
2021-10-15 17:10:00,6155.0,6155.0,6153.0,6155.0,31.0
2021-10-15 17:11:00,6155.0,6155.0,6153.0,6154.0,41.0
2021-10-15 17:12:00,6154.0,6154.0,6151.0,6152.0,20.0
...,...,...,...,...,...
2021-10-15 19:15:00,6188.0,6192.0,6186.0,6191.0,72.0
2021-10-15 19:16:00,6191.0,6191.0,6182.0,6183.0,237.0
2021-10-15 19:17:00,6183.0,6183.0,6176.0,6177.0,150.0


In [74]:
ohlc.loc['2021-10-15 18:41:00']

open           6131.0
high           6137.0
low            6127.0
close          6137.0
volume_diff      51.0
Name: 2021-10-15 18:41:00, dtype: float64